How to run this code:
Basically, we just follow the order of the cells(top to bottom)
(1) first import useful packages

(2) second import necessary root and path to download the corresponding Kaggle data set.

(3) build the cnn model based on differend parameters(input channels, output channels, pooling, activation, normalization method/or not)

(4) construct augmentation strategy,  define the train, validation, and test dataloader

(5) train model-> validate model by using Dev dataset -> make prediction on test data->output to kaggle.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as ttf

import os
import os.path as osp

from tqdm import tqdm
from PIL import Image
from sklearn.metrics import roc_auc_score
import numpy as np
!pip3 install ipdb
import ipdb

# TODOs
As you go, please read the code and keep an eye out for TODOs!

# Download Data

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"","key":""}') # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

     |████████████████████████████████| 59 kB 3.2 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73275 sha256=f90215b0d2725668673e113aadcd080a94140bb0fa41f4146f83a18407771b50
  Stored in directory: /root/.cache/pip/wheels/de/f7/d8/c3902cacb7e62cb611b1ad343d7cc07f42f7eb76ae3a52f3d1
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
from IPython.core.display import JSON
!pip install --upgrade --force-reinstall --no-deps kaggle
#!mkdir /root/.kaggle
import json
token = {"username":"shaofengqin","key":"f8b852f6077e65972f63df09c96f42d3"}
! pwd
with open('/root/.kaggle/kaggle.json', 'w') as file:
  json.dump(token,file)
for i in range(2):
  !chmod 600 /root/.kaggle/kaggle.json
  !cp /root/.kaggle/kaggle.json ~/.kaggle/
  !kaggle config set -n path -v /root
  !cat /root/.kaggle/kaggle.json

     |████████████████████████████████| 58 kB 3.2 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=ed41b1854ec24219363d33970ccf7a5e73e99238d8f924a8d6f24c827f115714
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.8
    Uninstalling kaggle-1.5.8:
      Successfully uninstalled kaggle-1.5.8
/content
cp: '/root/.kaggle/kaggle.json' and '/root/.kaggle/kaggle.json' are the same file
- path is now set to: /root
{
  "username": "shaofengqin",
  "key": "f8b852f6077e65972f63df09c96f42d3",
  "path": "/root"
}cp: '/root/.kaggle/kaggle.json' and '/root/.kaggle/kaggle.json' are the same file
- path is now set to: /root
{
  "username": "shaofengqin",
  "key": "f8b852f6077e65972f63df09c96f42d3",
  "path": "/root"
}

In [ ]:
!kaggle competitions download --force -c 11-785-s22-hw2p2-classification
!kaggle competitions download --force -c 11-785-s22-hw2p2-verification

!unzip /root/competitions/11-785-s22-hw2p2-classification/11-785-s22-hw2p2-classification.zip
!unzip /root/competitions/11-785-s22-hw2p2-verification/11-785-s22-hw2p2-verification.zip

!ls

Streaming output truncated to the last 5000 lines.
  inflating: verification/verification/test/cb17c2c0ac.jpg  
  inflating: verification/verification/test/cb17ded8b6.jpg  
  inflating: verification/verification/test/cb1a90fd77.jpg  
  inflating: verification/verification/test/cb1d81c2c7.jpg  
  inflating: verification/verification/test/cb1db575db.jpg  
  inflating: verification/verification/test/cb20065ce9.jpg  
  inflating: verification/verification/test/cb210d8d8e.jpg  
  inflating: verification/verification/test/cb24e086a1.jpg  
  inflating: verification/verification/test/cb29cd8d6a.jpg  
  inflating: verification/verification/test/cb2bcf5ad2.jpg  
  inflating: verification/verification/test/cb2f50e167.jpg  
  inflating: verification/verification/test/cb2f8b1174.jpg  
  inflating: verification/verification/test/cb3446ae16.jpg  
  inflating: verification/verification/test/cb347c38cd.jpg  
  inflating: verification/verification/test/cb39ad3570.jpg  
  inflating: verification/verifica

# Hyperparameters

Details about the experiments you ran, architectures you tried, number of epochs you trained for, hyperparameters you used (and specify which combination resulted in the best score), as well as your data loading scheme:

(1) The architectures I used are mainly Resnet 34 as given in the paepr, and I added a small modification by addingy one more residual block in the last part(originally 3 blocks, now I haev 4). Each residual block consists of two layers of 2D convolution and a short cut with ReLU activation. Each 2D convolution neural network is followed by a batchnorm layer, then by ReLU activation layer.

(2) I have tried several hyperparameters:
archetectures: 
1. three layers of conv2D, each followed by a batchnorm layer and RELU activation
2. Original Resnet 34, in addition to the initial conv2D layers(inchannel = 3, outchannel = 64), and pool(to 512), flatten layer, one linear classification layer.

3. Improved "resnet 34" by adding on ResidualBlock(in = 512,out = 512) --->preferred




input&output chanels: 32-64, 64-128, 128-256, 256-512(preferred), except for the input channel = 3 is fixed.

activation functions(RELU)

number of epochs(50, 60, 70(prefered),90), 

batchsize: 256(as suggested)

learning_rate: 0.1(as suggested), 0.05(tried, but not good)

Optimizer: 

1. SGD: SGD(model.parameters(), lr=lr, momentum=0.9, nesterov=True, weight_decay=5e-4) --> preferred, I tried momentum = 0.8, 0.7, nestrov = False.

2. Adam: Adam(model.parameters(), lr= lr, weight_decay=5e-4)

Scheduler:
1. ReduceLROnPlateau(optimizer, mode="max", factor=0.8, patience=4, threshold=0.001) ---> preferred

2. CosineAnnealingLR(optimizer, T_max=(len(train_loader) * epochs))

Criterion: CrossEntropyLoss(label_smoothing = 0.15(preferred), I also tried 0.1.0.15,0.2)

Data Augmentation: I tried several augumentation strategy(centercrop, color jitter, random rotate) but found auto augmentation gives the best result since it is leanred through reinenforcement, and a paper link can be found asL https://arxiv.org/pdf/1805.09501.pdf 




Data loading scheme:  Input images are directly imprted to the dataloader by using torchvision.datasets.ImageFolder(). And we apply the data augmentation transform when importing the data images. For face recognition task, the output of our prediction is recorded in a list(using armax to assign prediction on class, we called "classification_submission.csv", which is the file I submitted to kaggle. For face verification task, as suggested in the context code, I first used my trained model to predict the features/classes, and then use cosine similarity to compute similarity with two images and record them in a list, and compute AUC with labels for true similarities. Then we submit the "verification_submission.csv" that records the similarities for each pair of images to kaggle.


In [ ]:
"""
The well-accepted SGD batch_size & lr combination for CNN classification is 256 batch size for 0.1 learning rate.
When changing batch size for SGD, follow the linear scaling rule - halving batch size -> halve learning rate, etc.
This is less theoretically supported for Adam, but in my experience, it's a decent ballpark estimate.
"""
batch_size = 256
lr = 0.1
epochs = 70 # Just for the early submission. We'd want you to train like 50 epochs for your main submissions.

In [ ]:
# design a residue block
# each block consists of convnet and 
class ResidualBlock(nn.Module):
    def __init__(self, in_channel,out_channel, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channel)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(out_channel, out_channel, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channel)


        if stride == 1:
            self.shortcut = nn.Identity()
        else:
            self.shortcut = nn.Conv2d(in_channel, out_channel, kernel_size=1, stride=stride)
        self.relu = nn.ReLU()

    def forward(self, x):
        # print(np.shape(x))
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        out = self.conv2(out)
        out = self.bn2(out)
        shortcut = self.shortcut(x)
        # print(np.shape(out),np.shape(shortcut))
        out = self.relu(out + shortcut)

        return out


# Very Simple Network

In [ ]:
class Network(nn.Module):
    """
    The Very Low early deadline architecture is a 4-layer CNN.
    The first Conv layer has 64 channels, kernel size 7, and stride 4.
    The next three have 128, 256, and 512 channels. Each have kernel size 3 and stride 2.
    Think about what the padding should be for each layer to not change spatial resolution.
    Each Conv layer is accompanied by a Batchnorm and ReLU layer.
    Finally, you want to average pool over the spatial dimensions to reduce them to 1 x 1.
    Then, remove (Flatten?) these trivial 1x1 dimensions away.
    Look through https://pytorch.org/docs/stable/nn.html 
    TODO: Fill out the model definition below! 

    Why does a very simple network have 4 convolutions?
    Input images are 224x224. Note that each of these convolutions downsample.
    Downsampling 2x effectively doubles the receptive field, increasing the spatial
    region each pixel extracts features from. Downsampling 32x is standard
    for most image models.

    Why does a very simple network have high channel sizes?
    Every time you downsample 2x, you do 4x less computation (at same channel size).
    To maintain the same level of computation, you 2x increase # of channels, which 
    increases computation by 4x. So, balances out to same computation.
    Another intuition is - as you downsample, you lose spatial information. Want
    to preserve some of it in the channel dimension.
    """
    """
     # TODO: Conv group 2
            torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride = 2, padding= 1),
            nn.BatchNorm2d(128, eps=1e-05, momentum=0.1),
            nn.ReLU(),
            # TODO: Conv group 3
            torch.nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride = 2, padding= 1),
            nn.BatchNorm2d(256, eps=1e-05, momentum=0.1),
            nn.ReLU(),
            # TODO: Conv group 4
            torch.nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride = 2, padding= 1),
            nn.BatchNorm2d(512, eps=1e-05, momentum=0.1),
            nn.ReLU(),
            """
    def __init__(self, num_classes=7000):
        super().__init__()

        self.backbone = nn.Sequential(
            # Note that first conv is stride 4. It is (was?) standard to downsample.
            # 4x early on, as with 224x224 images, 4x4 patches are just low-level details.
            # Food for thought: Why is the first conv kernel size 7, not kernel size 3?

            # TODO: Conv group 1
            torch.nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride = 2, padding= 3),
            nn.BatchNorm2d(64, eps=1e-05, momentum=0.1),
            nn.ReLU(),
            nn.MaxPool2d(3, 2, 1),
            
            ResidualBlock(64,64), # *3
            ResidualBlock(64,64),
            ResidualBlock(64,64),

            ResidualBlock(64,128,stride=2), # *4
            ResidualBlock(128,128),
            ResidualBlock(128,128),
            ResidualBlock(128,128),

            ResidualBlock(128,256,stride=2),   # *6
            ResidualBlock(256,256),
            ResidualBlock(256,256),
            ResidualBlock(256,256),
            ResidualBlock(256,256),
            ResidualBlock(256,256),

            ResidualBlock(256,512,stride=2), # *4
            ResidualBlock(512,512),
            ResidualBlock(512,512),
            ResidualBlock(512,512),


            # TODO: Average pool over & reduce the spatial dimensions to (1, 1)
            nn.AdaptiveAvgPool2d((1,1)),
            # TODO: Collapse (Flatten) the trivial (1, 1) dimensions
            nn.Flatten())
        
        self.cls_layer = nn.Linear(512, num_classes)
    
    def forward(self, x, return_feats=False):
        """
        What is return_feats? It essentially returns the second-to-last-layer
        features of a given image. It's a "feature encoding" of the input image,
        and you can use it for the verification task. You would use the outputs
        of the final classification layer for the classification task.

        You might also find that the classification outputs are sometimes better
        for verification too - try both.
        """
        #ipdb.set_trace()
        feats = self.backbone(x)
        out = self.cls_layer(feats)

        if return_feats:
            return feats
        else:
            return out

# Dataset & DataLoader

# New Section

In [ ]:
"""
Transforms (data augmentation) is quite important for this task.
Go explore https://pytorch.org/vision/stable/transforms.html for more details
"""
DATA_DIR = "/content"
TRAIN_DIR = osp.join(DATA_DIR, "classification/classification/train") # This is a smaller subset of the data. Should change this to classification/classification/train
VAL_DIR = osp.join(DATA_DIR, "classification/classification/dev")
TEST_DIR = osp.join(DATA_DIR, "classification/classification/test")

train_transforms = [ttf.AutoAugment(), ttf.ToTensor()]
val_transforms = [ttf.ToTensor()]

train_dataset = torchvision.datasets.ImageFolder(TRAIN_DIR,
                                                 transform=ttf.Compose(train_transforms))
val_dataset = torchvision.datasets.ImageFolder(VAL_DIR,
                                               transform=ttf.Compose(val_transforms))


train_loader = DataLoader(train_dataset, batch_size=batch_size,
                          shuffle=True, drop_last=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                        drop_last=True, num_workers=1)

# Setup everything for training

In [ ]:
from torch.optim import optimizer
model = Network()
model.cuda()

# For this homework, we're limiting you to 35 million trainable parameters, as
# outputted by this. This is to help constrain your search space and maintain
# reasonable training times & expectations
num_trainable_parameters = 0
for p in model.parameters():
    num_trainable_parameters += p.numel()
print("Number of Params: {}".format(num_trainable_parameters))

# TODO: What criterion do we use for this task?
criterion = torch.nn.CrossEntropyLoss(label_smoothing = 0.15)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, nesterov=True, weight_decay=5e-4)
#optimizer = optim.Adam(model.parameters(), lr= lr, weight_decay=5e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", factor=0.8, patience=4, threshold=0.001)
#scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(len(train_loader) * epochs))
# T_max is "how many times will i call scheduler.step() until it reaches 0 lr?"

# For this homework, we strongly strongly recommend using FP16 to speed up training.
# It helps more for larger models.
# Go to https://effectivemachinelearning.com/PyTorch/8._Faster_training_with_mixed_precision
# and compare "Single precision training" section with "Mixed precision training" section
scaler = torch.cuda.amp.GradScaler()

Number of Params: 29595480


# Let's train!

In [ ]:
def validation(model, val_loader):
    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')
    num_correct = 0
    for i, (x, y) in enumerate(val_loader):
        x = x.cuda()
        y = y.cuda()
        with torch.no_grad():
            outputs = model(x)
        num_correct += int((torch.argmax(outputs, axis=1) == y).sum())
        batch_bar.set_postfix(acc="{:.04f}%".format(100 * num_correct / ((i + 1) * batch_size)))
        batch_bar.update()
    batch_bar.close()
    acc = 100 * num_correct / len(val_dataset)
    return acc

In [ ]:
for epoch in range(epochs):
    # Quality of life tip: leave=False and position=0 are needed to make tqdm usable in jupyter
    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 

    num_correct = 0
    total_loss = 0

    for i, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()

        x = x.cuda()
        y = y.cuda()

        # Don't be surprised - we just wrap these two lines to make it work for FP16
        with torch.cuda.amp.autocast():     
            outputs = model(x)
            loss = criterion(outputs, y)

        # Update # correct & loss as we go
        num_correct += int((torch.argmax(outputs, axis=1) == y).sum())
        total_loss += float(loss)

        # tqdm lets you add some details so you can monitor training as you train.
        batch_bar.set_postfix(
            acc="{:.04f}%".format(100 * num_correct / ((i + 1) * batch_size)),
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            num_correct=num_correct,
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        
        # Another couple things you need for FP16. 
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        #scheduler.step() # This the step for using CosineAnnealingLR. We setp in each batch. We told scheduler T_max that we'd call step() (len(train_loader) * epochs) many times.

        batch_bar.update() # Update tqdm bar
    eval_acc = validation(model, val_loader)
    scheduler.step(eval_acc)
    print("Epoch {}/{}: evaluation Acc {:.04f}%".format(
        epoch + 1,
        epochs,
        eval_acc))
    batch_bar.close() # You need this to close the tqdm bar

    # You can add validation per-epoch here if you would like

    print("Epoch {}/{}: Train Acc {:.04f}%, Train Loss {:.04f}, Learning Rate {:.04f}".format(
        epoch + 1,
        epochs,
        100 * num_correct / (len(train_loader) * batch_size),
        float(total_loss / len(train_loader)),
        float(optimizer.param_groups[0]['lr'])))

Epoch 1/70: evaluation Acc 0.0143%
Epoch 1/70: Train Acc 0.0122%, Train Loss nan, Learning Rate 0.1000


Epoch 2/70: evaluation Acc 0.0143%
Epoch 2/70: Train Acc 0.0114%, Train Loss 8.8866, Learning Rate 0.1000


Epoch 3/70: evaluation Acc 0.0143%
Epoch 3/70: Train Acc 0.0136%, Train Loss 8.8863, Learning Rate 0.1000


Epoch 4/70: evaluation Acc 0.0143%
Epoch 4/70: Train Acc 0.0114%, Train Loss 8.8854, Learning Rate 0.1000


Epoch 5/70: evaluation Acc 0.0143%
Epoch 5/70: Train Acc 0.0100%, Train Loss 8.8855, Learning Rate 0.1000


Epoch 6/70: evaluation Acc 0.0143%
Epoch 6/70: Train Acc 0.0100%, Train Loss 8.8868, Learning Rate 0.0800


Epoch 7/70: evaluation Acc 0.0143%
Epoch 7/70: Train Acc 0.0100%, Train Loss 8.8803, Learning Rate 0.0800


Epoch 8/70: evaluation Acc 0.0143%
Epoch 8/70: Train Acc 0.0129%, Train Loss 8.8806, Learning Rate 0.0800


Train:  20%|██        | 110/546 [01:11<04:38,  1.57it/s, acc=0.0176%, loss=8.8798, lr=0.0800, num_correct=5]

KeyboardInterrupt: ignored

# Classification Task: Validation

In [ ]:
model.eval()
batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')
num_correct = 0
for i, (x, y) in enumerate(val_loader):

    x = x.cuda()
    y = y.cuda()

    with torch.no_grad():
        outputs = model(x)

    num_correct += int((torch.argmax(outputs, axis=1) == y).sum())
    batch_bar.set_postfix(acc="{:.04f}%".format(100 * num_correct / ((i + 1) * batch_size)))

    batch_bar.update()
    
batch_bar.close()
print("Validation: {:.04f}%".format(100 * num_correct / len(val_dataset)))

Validation: 82.7657%


# Classification Task: Submit to Kaggle

In [ ]:
class ClassificationTestSet(Dataset):
    # It's possible to load test set data using ImageFolder without making a custom class.
    # See if you can think it through!

    def __init__(self, data_dir, transforms):
        self.data_dir = data_dir
        self.transforms = transforms

        # This one-liner basically generates a sorted list of full paths to each image in data_dir
        self.img_paths = list(map(lambda fname: osp.join(self.data_dir, fname), sorted(os.listdir(self.data_dir))))

    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        return self.transforms(Image.open(self.img_paths[idx]))

In [ ]:
test_dataset = ClassificationTestSet(TEST_DIR, ttf.Compose(val_transforms))
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                         drop_last=False, num_workers=1)

In [ ]:
model.eval()
batch_bar = tqdm(total=len(test_loader), dynamic_ncols=True, position=0, leave=False, desc='Test')

res = []
for i, (x) in enumerate(test_loader):

    # TODO: Finish predicting on the test set.
    x = x.cuda()
    with torch.no_grad():
        outputs = model(x)
    pred_y = torch.argmax(outputs, axis=1)
    res.extend(pred_y.tolist())
    

    batch_bar.update()
    
batch_bar.close()

In [ ]:
with open("classification_submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(test_dataset)):
        f.write("{},{}\n".format(str(i).zfill(6) + ".jpg", res[i]))

In [ ]:
! kaggle competitions submit -c face-recognition-slack -f classification_submission.csv -m "Message"

100% 541k/541k [00:07<00:00, 72.6kB/s]
Successfully submitted to Face Recognition (Slack)

# Verification Task: Validation

There are 6K verification dev images, but 166K "pairs" for you to compare. So, it's much more efficient to compute the features for the 6K verification images, and just compare afterwards.

This will be done by creating a dictionary mapping the image file names to the features. Then, you'll use this dictionary to compute the similarities for each pair.

In [ ]:
!ls verification/verification/dev | wc -l
!cat verification/verification/verification_dev.csv | wc -l

6000
166801


In [ ]:
class VerificationDataset(Dataset):
    def __init__(self, data_dir, transforms):
        self.data_dir = data_dir
        self.transforms = transforms

        # This one-liner basically generates a sorted list of full paths to each image in data_dir
        self.img_paths = list(map(lambda fname: osp.join(self.data_dir, fname), sorted(os.listdir(self.data_dir))))

    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        # We return the image, as well as the path to that image (relative path)
        return self.transforms(Image.open(self.img_paths[idx])), osp.relpath(self.img_paths[idx], self.data_dir)

In [ ]:
val_veri_dataset = VerificationDataset(osp.join(DATA_DIR, "verification/verification/dev"),
                                       ttf.Compose(val_transforms))
val_ver_loader = torch.utils.data.DataLoader(val_veri_dataset, batch_size=batch_size, 
                                             shuffle=False, num_workers=1)

In [ ]:
model.eval()

feats_dict = dict()
for batch_idx, (imgs, path_names) in tqdm(enumerate(val_ver_loader), total=len(val_ver_loader), position=0, leave=False):
    imgs = imgs.cuda()

    with torch.no_grad():
        # Note that we return the feats here, not the final outputs
        # Feel free to try the final outputs too!
        feats = model(imgs, return_feats=True) 
    
    # TODO: Now we have features and the image path names. What to do with them?
    # Hint: use the feats_dict somehow.
    for i, (feature) in enumerate(feats):
      feats_dict[path_names[i]] = feature

In [ ]:
# What does this dict look like?
print(list(feats_dict.items())[0])

('000b28b024.jpg', tensor([7.4789e-01, 4.2560e-01, 6.1766e-01, 1.1123e+00, 4.0175e+00, 2.1440e-01,
        1.3493e+00, 3.3815e-02, 2.0557e-01, 9.2872e-01, 2.1153e+00, 2.3474e-01,
        1.7418e-01, 2.5076e-01, 4.4688e-01, 3.6740e-01, 3.6967e+00, 6.2941e-04,
        4.1166e-01, 7.2265e-01, 3.3956e-01, 2.8071e+00, 2.8456e-01, 1.0969e+00,
        3.9593e+00, 0.0000e+00, 5.4472e+00, 6.0907e-02, 1.4188e-01, 8.9241e-01,
        0.0000e+00, 1.6200e+00, 4.6475e-01, 1.0988e+00, 3.3201e-01, 1.9135e-02,
        1.4587e+00, 2.5842e+00, 1.1929e+00, 1.2091e-01, 2.9116e-01, 3.5917e+00,
        1.4947e+00, 3.5282e-01, 1.3198e+00, 4.7912e-01, 0.0000e+00, 1.3818e-01,
        9.8094e-02, 2.5035e+00, 2.5893e-01, 4.2576e-02, 9.5741e-01, 7.3929e+00,
        8.8301e-05, 1.4640e-02, 3.1001e-01, 1.7502e-01, 6.6818e-01, 8.2889e-02,
        2.4586e-01, 1.2522e+00, 1.2538e+00, 2.6151e-01, 4.4392e-01, 9.5594e-01,
        5.7674e-01, 3.6550e-01, 2.3343e-01, 4.3068e-01, 0.0000e+00, 2.7113e+00,
        1.2124e-03, 1

In [ ]:
# We use cosine similarity between feature embeddings.
# TODO: Find the relevant function in pytorch and read its documentation.
# similarity_metric = nn.functional.cosine_similarity()

val_veri_csv = osp.join(DATA_DIR, "verification/verification/verification_dev.csv")


# Now, loop through the csv and compare each pair, getting the similarity between them
pred_similarities = []
gt_similarities = []
for line in tqdm(open(val_veri_csv).read().splitlines()[1:], position=0, leave=False): # skip header
    img_path1, img_path2, gt = line.split(",")

    # TODO: Use the similarity metric
    # How to use these img_paths? What to do with the features?
    similarity = nn.functional.cosine_similarity(feats_dict.get(img_path1.split('/')[1]), feats_dict.get(img_path2.split('/')[1]),dim = 0 ).cpu()
    pred_similarities.append(similarity)
    gt_similarities.append(int(gt))

pred_similarities = np.array(pred_similarities)
gt_similarities = np.array(gt_similarities)

print("AUC:", roc_auc_score(gt_similarities, pred_similarities))

AUC: 0.9543623101170993


# Verification Task: Submit to Kaggle

In [ ]:
test_veri_dataset = VerificationDataset(osp.join(DATA_DIR, "verification/verification/test"),
                                        ttf.Compose(val_transforms))
test_ver_loader = torch.utils.data.DataLoader(test_veri_dataset, batch_size=batch_size, 
                                              shuffle=False, num_workers=1)

In [ ]:
model.eval()

feats_dict = dict()
for batch_idx, (imgs, path_names) in tqdm(enumerate(test_ver_loader), total=len(test_ver_loader), position=0, leave=False):
    imgs = imgs.cuda()

    with torch.no_grad():
        # Note that we return the feats here, not the final outputs
        # Feel free to try to final outputs too!
        feats = model(imgs, return_feats=True) 
    
    # TODO: Now we have features and the image path names. What to do with them?
    # Hint: use the feats_dict somehow.
    for i, (feature) in enumerate(feats):
      feats_dict[path_names[i]] = feature


In [ ]:
# We use cosine similarity between feature embeddings.
# TODO: Find the relevant function in pytorch and read its documentation.
# similarity_metric = nn.functional.cosine_similarity()
val_veri_csv = osp.join(DATA_DIR, "verification/verification/verification_test.csv")


# Now, loop through the csv and compare each pair, getting the similarity between them
pred_similarities = []
gt_similarities = []
for line in tqdm(open(val_veri_csv).read().splitlines()[1:], position=0, leave=False): # skip header
    img_path1, img_path2 = line.split(",")
    similarity = nn.functional.cosine_similarity(feats_dict.get(img_path1.split('/')[1]), feats_dict.get(img_path2.split('/')[1]),dim = 0 ).cpu()
    pred_similarities.append(similarity)
    gt_similarities.append(int(gt))

    # TODO: Finish up verification testing.
    # How to use these img_paths? What to do with the features?

In [ ]:
with open("verification_submission.csv", "w+") as f:
    f.write("id,match\n")
    for i in range(len(pred_similarities)):
        f.write("{},{}\n".format(i, pred_similarities[i]))

In [ ]:
!kaggle competitions submit -c face-verification-slack -f verification_submission.csv -m "Message"

100% 16.6M/16.6M [00:03<00:00, 4.47MB/s]
Successfully submitted to Face Verification (Slack)

# Extras

In [ ]:
# If you keep re-initializing your model in Colab, can run out of GPU memory, need to restart.
# These three lines can help that - run this before you re-initialize your model

del model
torch.cuda.empty_cache()
!nvidia-smi

NameError: ignored

record the first trial

In [ ]:
torch.save(model.state_dict(),'test 1')